In [3]:
!pip install pandas numpy scikit-learn scipy surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505210 sha256=5d7a28168dcb9360b071f0114248b4d8c89c33f623bdac6cefaacd287a172a45
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [12]:
import pandas as pd
import numpy as np

# Assuming your files are uploaded in the Colab environment's content folder
ratings = pd.read_csv("/ratings.csv")
movies = pd.read_csv("/movies.csv")

print("Ratings data:")
print(ratings.head())
print("\nMovies data:")
print(movies.head())

Ratings data:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Movies data:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [13]:
user_item_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
print("User-Item Matrix Shape:", user_item_matrix.shape)

User-Item Matrix Shape: (610, 9724)


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

print("\nUser-User Similarity Matrix:")
print(user_similarity_df.head())


User-User Similarity Matrix:
userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2       0.027283  1.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  1.000000  0.002251  0.005020  0.003936  0.000000   
4       0.194395  0.003726  0.002251  1.000000  0.128659  0.088491  0.115120   
5       0.129080  0.016614  0.005020  0.128659  1.000000  0.300349  0.108342   

userId       8         9         10   ...       601       602       603  \
userId                                ...                                 
1       0.136968  0.064263  0.016875  ...  0.080554  0.164455  0.221486   
2       0.027257  0.000000  0.067445  ...  0.202671  0.016866  0.011997   
3       0.004941  0.000000  0.000000  ...  0.005048  0.004892  0.024992   
4       0.062969  0.011361  0.0311

In [16]:
def recommend_movies(user_id, n_recommendations=5):

    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:11]

    similar_users_ratings = user_item_matrix.loc[similar_users.index]

    weighted_ratings = similar_users_ratings.mul(similar_users, axis=0).sum() / similar_users.sum()

    user_rated = user_item_matrix.loc[user_id] > 0
    recommendations = weighted_ratings[~user_rated].sort_values(ascending=False)

    return movies.set_index('movieId').loc[recommendations.head(n_recommendations).index]

print("\nTop 5 Recommendations for User 1:")
print(recommend_movies(1))


Top 5 Recommendations for User 1:
                                     title                          genres
movieId                                                                   
589      Terminator 2: Judgment Day (1991)                   Action|Sci-Fi
1200                         Aliens (1986)  Action|Adventure|Horror|Sci-Fi
2762               Sixth Sense, The (1999)            Drama|Horror|Mystery
1610      Hunt for Red October, The (1990)       Action|Adventure|Thriller
858                  Godfather, The (1972)                     Crime|Drama
